In [1]:
%load_ext autoreload
%autoreload 2

# [Streaming](https://python.langchain.com/docs/how_to/streaming/)

In [2]:
def load_env_to_dict(file_path):
    env_dict = {}
    with open(file_path, "r") as file:
        for line in file:
            # Remove whitespace and ignore comments or empty lines
            line = line.strip()
            if not line or line.startswith("#"):
                continue
            # Split the line into key and value
            key, value = line.split("=", 1)
            env_dict[key.strip()] = value.strip()
    return env_dict

In [3]:
file_path = "/mnt/Exdisk/git-cuongpiger/secret/work/vngcloud/ai-platform/env"
env_variables = load_env_to_dict(file_path)

In [4]:
import os

if not os.getenv("HUGGINGFACEHUB_API_TOKEN"):
    os.environ["HUGGINGFACEHUB_API_TOKEN"] = env_variables["HUGGINGFACE_API_KEY"]

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = (
    "/mnt/Exdisk/git-cuongpiger/secret/work/vngcloud/ai-platform/vertex-ai-credential.json"
)

In [5]:
from langchain_google_vertexai import ChatVertexAI

In [8]:
model = ChatVertexAI(model="gemini-1.5-flash")

In [9]:
chunks = []
for chunk in model.stream("Viết cho tôi một câu tỏ tình ngọt ngào"):
    chunks.append(chunk)
    print(chunk.content, end="|", flush=True)

Em| biết không, mỗi khi em cười, thế giới như sáng bừng lên,| và em chính là ánh nắng ấm áp trong trái tim anh. Anh muốn được| là người mang lại nụ cười ấy mãi mãi, được ở bên cạnh em, cùng em tạo nên những kỉ niệm đẹp. Em có muốn cho anh| cơ hội được yêu thương và chăm sóc em không? 
|

In [10]:
chunks = []
async for chunk in model.astream("Viết cho tôi một câu tỏ tình ngọt ngào"):
    chunks.append(chunk)
    print(chunk.content, end="|", flush=True)

Em| biết không, mỗi khi em cười, trái tim anh như muốn nhảy múa|. Em tỏa sáng như một vì sao, và anh muốn được là bầu| trời bao bọc em. Anh yêu em, và mong muốn được cùng em bước đi trên con đường hạnh phúc. Em có muốn đồng hành cùng anh không?| 💕
|

In [11]:
chunks[0]

AIMessageChunk(content='Em', additional_kwargs={}, response_metadata={'safety_ratings': []}, id='run-27e7f624-33a5-40cc-9d27-8252071b2008')

In [12]:
chunks[0] + chunks[1] + chunks[2] + chunks[3] + chunks[4]

AIMessageChunk(content='Em biết không, mỗi khi em cười, trái tim anh như muốn nhảy múa. Em tỏa sáng như một vì sao, và anh muốn được là bầu trời bao bọc em. Anh yêu em, và mong muốn được cùng em bước đi trên con đường hạnh phúc. Em có muốn đồng hành cùng anh không? 💕\n', additional_kwargs={}, response_metadata={'safety_ratings': [{'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability_label': 'NEGLIGIBLE', 'probability_score': 0.13753287494182587, 'blocked': False, 'severity': 'HARM_SEVERITY_NEGLIGIBLE', 'severity_score': 0.09947013854980469}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability_label': 'NEGLIGIBLE', 'probability_score': 0.033589545637369156, 'blocked': False, 'severity': 'HARM_SEVERITY_NEGLIGIBLE', 'severity_score': 0.07369651645421982}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability_label': 'NEGLIGIBLE', 'probability_score': 0.12085322290658951, 'blocked': False, 'severity': 'HARM_SEVERITY_NEGLIGIBLE', 'severity_score': 0.11279552429914474}, {'category'

In [13]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

In [14]:
prompt = ChatPromptTemplate.from_template("Kể cho tôi một câu chuyện cười về {topic}")
parser = StrOutputParser()
chain = prompt | model | parser

In [17]:
async for chunk in chain.astream({"topic": "gia đình"}):
    print(chunk, end="|", flush=True)

Một| cậu bé hỏi bố: "Bố ơi, con có phải là được sinh ra| từ trái đất không ạ?"

Bố cười: "Không con yêu, con| được sinh ra từ bệnh viện."

Cậu bé suy nghĩ một chút rồi hỏi: "Vậy tại sao mẹ con lại cứ nói mẹ sinh con ở| trái đất?" 

Bố bật cười: "À, đó là cách nói vui thôi con. Mẹ con chỉ muốn con biết con được sinh ra từ| một nơi rất đặc biệt."

Cậu bé gật gù, rồi hỏi tiếp: "Vậy bố có biết trái đất ở đâu không?"

Bố trả lời: "Chắc chắn rồi con! Trái đất ở| ngay trong tủ lạnh!"

Cậu bé ngạc nhiên: "Sao bố lại biết? Sao bố không lấy trái đất ra cho con xem?"

Bố đáp: "Con à, nếu bố lấy trái đất ra thì bố mẹ sẽ| không có chỗ để đựng thức ăn nữa!"
|

In [18]:
from langchain_core.output_parsers import JsonOutputParser

In [19]:
chain = (
    model | JsonOutputParser()
)  # Due to a bug in older versions of Langchain, JsonOutputParser did not stream results from some models

In [20]:
async for text in chain.astream(
    "output a list of the countries france, spain and japan and their populations in JSON format. "
    'Use a dict with an outer key of "countries" which contains a list of countries. '
    "Each country should have the key `name` and `population`"
):
    print(text, flush=True)

{}
{'countries': [{'name': 'France'}]}
{'countries': [{'name': 'France', 'population': 67.39}]}
{'countries': [{'name': 'France', 'population': 67.39}, {'name': 'Spain', 'population': 47.35}, {}]}
{'countries': [{'name': 'France', 'population': 67.39}, {'name': 'Spain', 'population': 47.35}, {'name': 'Japan', 'population': 125.8}]}


In [21]:
from langchain_core.output_parsers import (
    JsonOutputParser,
)

In [22]:
# A function that operates on finalized inputs
# rather than on an input_stream
def _extract_country_names(inputs):
    """A function that does not operates on input streams and breaks streaming."""
    if not isinstance(inputs, dict):
        return ""

    if "countries" not in inputs:
        return ""

    countries = inputs["countries"]

    if not isinstance(countries, list):
        return ""

    country_names = [
        country.get("name") for country in countries if isinstance(country, dict)
    ]
    return country_names

In [23]:
chain = model | JsonOutputParser() | _extract_country_names

In [24]:
async for text in chain.astream(
    "output a list of the countries france, spain and japan and their populations in JSON format. "
    'Use a dict with an outer key of "countries" which contains a list of countries. '
    "Each country should have the key `name` and `population`"
):
    print(text, end="|", flush=True)

['France', 'Spain', 'Japan']|

In [25]:
from langchain_core.output_parsers import JsonOutputParser

In [26]:
async def _extract_country_names_streaming(input_stream):
    """A function that operates on input streams."""
    country_names_so_far = set()

    async for input in input_stream:
        if not isinstance(input, dict):
            continue

        if "countries" not in input:
            continue

        countries = input["countries"]

        if not isinstance(countries, list):
            continue

        for country in countries:
            name = country.get("name")
            if not name:
                continue
            if name not in country_names_so_far:
                yield name
                country_names_so_far.add(name)

In [27]:
chain = model | JsonOutputParser() | _extract_country_names_streaming

In [28]:
async for text in chain.astream(
    "output a list of the countries france, spain and japan and their populations in JSON format. "
    'Use a dict with an outer key of "countries" which contains a list of countries. '
    "Each country should have the key `name` and `population`",
):
    print(text, end="|", flush=True)

France|Spain|Japan|

# Non-streaming components

In [31]:
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_google_vertexai import ChatVertexAI, VertexAIEmbeddings

In [34]:
template = """Trả lời cầu dưới và chỉ dựa trên những thông tin dưới đây:
{context}

Câu hỏi: {question}
"""

In [35]:
prompt = ChatPromptTemplate.from_template(template)

In [39]:
vectorstore = FAISS.from_texts(
    ["Cường là một kỹ sư phần mềm", "Cường hiện đang làm việc tại VngCloud ở quận 7"],
    embedding=VertexAIEmbeddings(model_name="text-embedding-004"),
)

In [40]:
retriever = vectorstore.as_retriever()

In [41]:
chunks = [chunk for chunk in retriever.stream("Cường làm ở đâu")]
chunks

[[Document(id='11bed248-ff2b-4e16-bbe2-ce721fe33fa1', metadata={}, page_content='Cường là một kỹ sư phần mềm'),
  Document(id='5da732b7-7da4-4649-9d27-c9b6d95c6b1f', metadata={}, page_content='Cường hiện đang làm việc tại VngCloud ở quận 7')]]

In [42]:
retrieval_chain = (
    {
        "context": retriever.with_config(run_name="Docs"),
        "question": RunnablePassthrough(),
    }
    | prompt
    | model
    | StrOutputParser()
)

In [43]:
for chunk in retrieval_chain.stream(
    "Cường làm ở đâu" "Viết 3 câu về nơi làm việc của Cường"
):
    print(chunk, end="|", flush=True)

C|ường làm| việc tại VngCloud.
VngCloud là một công ty ở quận 7|. 
Công việc của Cường tại VngCloud là một kỹ sư phần mềm|. 
|